In [ ]:
%matplotlib inline

Solving quadratic problems - advanced
=====================================

::: {.seealso}
See
`sphx_glr_auto_examples_solvers_ex_quad_problems_basic.py`{.interpreted-text
role="ref"} for an introduction to solving quadratic problems. See
`sphx_glr_auto_examples_solvers_ex_linear_problems_basic.py`{.interpreted-text
role="ref"} for an introduction to solving linear problems. See
`sphx_glr_auto_examples_solvers_ex_linear_problems_advanced.py`{.interpreted-text
role="ref"} for an advanced example how to solve linear problems.
:::


In [ ]:
from moscot.datasets import simulate_data
from moscot.problems.generic import GWProblem

This examples is a continuation of TODO reference and shows advanced
examples for how to solve quadratic problems, e.g. the
`moscot.problems.time.LineageProblem`{.interpreted-text role="class"},
the
`moscot.problems.spatio_temporal.SpatioTemporalProblem`{.interpreted-text
role="class"}, the
`moscot.problems.space.MappingProblem`{.interpreted-text role="class"},
the `moscot.problems.time.AlignmentProblem`{.interpreted-text
role="class"}, the `moscot.problems.generic.GWProblem`{.interpreted-text
role="class"}, and the the
`moscot.problems.generic.FGWProblem`{.interpreted-text role="class"}.


In [ ]:
import scanpy as sc

adata = simulate_data(n_distributions=2, key="batch", quad_term="spatial")
sc.pp.pca(adata)
gwp = GWProblem(adata)
gwp = gwp.prepare(key="batch", GW_x={"attr": "obsm", "key": "spatial"}, GW_y={"attr": "obsm", "key": "spatial"})
adata

Threshold
=========

The [threshold]{.title-ref} parameter defines the convergence criterion.
In the balanced setting the [threshold]{.title-ref} denotes the
deviation between prior and posterior marginals, while in the unbalanced
setting the [threshold]{.title-ref} corresponds to a Cauchy sequence
stopping criterion.


Initializers
============

Different Initializers can help to improve convergence. For the
full-rank case only the default initializer exists, hence the
[initializer]{.title-ref} argument must be set to [None]{.title-ref}.

For low-rank problems the same initializers as for the linear low-rank
solvers are available, and [initializer\_kwargs]{.title-ref} can be
passed the same way, see TODO


Number of iterations
====================

To solve a quadratic Optimal Transport problem, a consecutively updated
linearized problem is solved [n\_iterations]{.title-ref} time. Here,
[min\_iterations]{.title-ref} denotes a lower bound for
[n\_iterations]{.title-ref} and [max\_iterations]{.title-ref} an upper
bound. If [max\_iterations]{.title-ref} is too low, the model might not
converge.


In [ ]:
gwp = gwp.solve(alpha=0.5, epsilon=1e-1, min_iterations=0, max_iterations=1)

Iterations of the inner loop & linear solver keyword arguments
==============================================================

As mentioned above, each outer loop step of the Gromov-Wasserstein
algorithm consists of solving a linear problem. Arguments for the linear
solver can be specified via [linear\_solver\_kwargs]{.title-ref},
keyword arguments for `ott.core.sinkhorn.Sinkhorn`{.interpreted-text
role="class"} in the full-rank case or keyword arguments for
`ott.core.sinkhorn_lr.LRSinkhorn`{.interpreted-text role="class"},
respectively. This way, we can also set the minimum and maximum number
of iterations for the linear solver:


In [ ]:
ls_kwargs = {"min_iterations": 10, "max_iterations": 1000, "threshold": 0.01}
gwp = gwp.solve(
    alpha=0.5, epsilon=1e-1, threshold=0.1, min_iterations=2, max_iterations=20, linear_solver_kwargs=ls_kwargs
)

Low rank hyperparameters
========================

The parameters [gamma]{.title-ref} and [gamma\_rescale]{.title-ref} are
the same as in the linear case, see example TODO. It remains to consider
[ranks]{.title-ref} and [tolerances]{.title-ref}.


Keyword arguments & Implementation details
==========================================

Whenever the [solve]{.title-ref} method of a quadratic problem is
called, a backend-specific quadratic solver is instantiated. Currently,
`ott`{.interpreted-text role="mod"} is supported, its corresponding
quadratic solvers is
`ott.core.gromov_wasserstein.GromovWasserstein`{.interpreted-text
role="class"}, handling both the full-rank and the low-rank case.
`moscot`{.interpreted-text role="mod"} wraps this class in
`moscot.backends.ott.GWSolver`{.interpreted-text role="class"} and
`moscot.backends.ott.FGWSolver`{.interpreted-text role="class"},
handling the purely quadratic and the fused quadratic problem,
respectively.
